In [120]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [121]:
# ZENTRALE PARAMETER (einfach hier ändern)

ARZT_ID = 1
PFLEGEKRAFT_ID = 1
KRANKENHAUS_ID = 1
TODAY = '2025-12-12'

def execute_print_query(cursor,query):
    cursor.execute(query)
    column_names = [desc[0] for desc in cur.description]
    print(column_names)
    for row in cur.fetchall():
        print(row)


Dr. Luc Thommen kommt zur Arbeit und möchte wissen, welche Patienten er heute betreut und wo sie sich befinden.

In [122]:
query = f"""
-- Zeigt alle heutigen Patienten eines Arztes inklusive Station und Event
SELECT
    p.patient_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    s.name AS station,
    e.datum,
    e.beschreibung AS event_beschreibung
FROM arzt a
JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
JOIN event e ON ae.event_id = e.event_id
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN station s ON p.station_id = s.station_id
WHERE a.arzt_id = {ARZT_ID}
  AND e.datum = DATE '{TODAY}'
ORDER BY s.name, patient_name;
"""
execute_print_query(cur, query)


['patient_id', 'patient_name', 'station', 'datum', 'event_beschreibung']
(63, 'Mato Winkler', 'Intensivstation', datetime.date(2025, 12, 12), 'Visite')
(41, 'Tanja Bernasconi', 'Notfall', datetime.date(2025, 12, 12), 'Visite')
(41, 'Tanja Bernasconi', 'Notfall', datetime.date(2025, 12, 12), 'Visite')
(6, 'Till Iten', 'Notfall', datetime.date(2025, 12, 12), 'Visite')
(28, 'Gordana Bättig', 'Pädiatrie', datetime.date(2025, 12, 12), 'Visite')


Der Arzt kontrolliert, welche Behandlungen noch nicht abgeschlossen sind.

In [123]:
query = """
SELECT
    b.behandlungs_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    b.beschreibung,
    b.start_datum
FROM arzt a
JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
JOIN event e ON ae.event_id = e.event_id
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
WHERE a.arzt_id = %s
  AND b.abgeschlossen = false
GROUP BY b.behandlungs_id, p.vorname, p.nachname, b.beschreibung, b.start_datum
ORDER BY b.start_datum;
"""
execute_print_query(cur, cur.mogrify(query, (ARZT_ID,)).decode())


['behandlungs_id', 'patient_name', 'beschreibung', 'start_datum']
(8, 'Tanja Bernasconi', 'Diabetes Verlaufskontrolle', datetime.date(2025, 12, 21))
(172, 'Tanja Bernasconi', 'Schmerztherapie', datetime.date(2026, 1, 7))


Der Arzt möchte wissen, welche Medikamente seine Patienten erhalten.

In [124]:
query = """
SELECT DISTINCT
    p.vorname || ' ' || p.nachname AS patient_name,
    m.name AS medikament,
    m.wirkstoff
FROM arzt a
JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
JOIN event e ON ae.event_id = e.event_id
JOIN event_medikament em ON e.event_id = em.event_id
JOIN medikament m ON em.medikament_id = m.medikament_id
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
WHERE a.arzt_id = %s
ORDER BY patient_name, medikament;
"""
execute_print_query(cur, cur.mogrify(query, (ARZT_ID,)).decode())


['patient_name', 'medikament', 'wirkstoff']
('Gordana Bättig', 'Ibuprofen', 'Ibuprofen')
('Gordana Bättig', 'Paracetamol', 'Paracetamol')
('Mato Winkler', 'Insulin', 'Insulin')
('Mato Winkler', 'Paracetamol', 'Paracetamol')
('Tanja Bernasconi', 'Ibuprofen', 'Ibuprofen')
('Tanja Bernasconi', 'Insulin', 'Insulin')
('Tanja Bernasconi', 'Paracetamol', 'Paracetamol')
('Till Iten', 'Aspirin', 'Acetylsalicylsäure')
('Till Iten', 'Paracetamol', 'Paracetamol')


Die Pflegekraft sieht alle Patienten auf ihrer Station.

In [125]:
query = """
SELECT
    pk.vorname || ' ' || pk.nachname AS pflegekraft,
    p.patient_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    s.name AS station
FROM pflegekraft pk
JOIN station s ON pk.station_id = s.station_id
JOIN patient p ON p.station_id = s.station_id
WHERE pk.pflege_id = %s
ORDER BY patient_name;
"""
execute_print_query(cur, cur.mogrify(query, (PFLEGEKRAFT_ID,)).decode())


['pflegekraft', 'patient_id', 'patient_name', 'station']


Die Pflegekraft kontrolliert, welche Events heute auf ihrer Station stattfinden.

In [126]:
query = """
SELECT
    e.event_id,
    e.uhrzeit,
    e.beschreibung,
    r.art AS raumart
FROM pflegekraft pk
JOIN station s ON pk.station_id = s.station_id
JOIN raum r ON r.station_id = s.station_id
JOIN event e ON e.raumid = r.raumnummer
WHERE pk.pflege_id = %s
  AND e.datum = %s
ORDER BY e.uhrzeit;
"""
execute_print_query(cur, cur.mogrify(query, (PFLEGEKRAFT_ID, DATUM)).decode())


['event_id', 'uhrzeit', 'beschreibung', 'raumart']


In [127]:
# Query 5: Zeigt alle Events des heutigen Tages,
# sortiert nach Raum und Uhrzeit (z. B. für Tagesplanung).

query = """
SELECT
    r.art AS raum_art,
    r.raumnummer,
    e.uhrzeit,
    p.vorname || ' ' || p.nachname AS patient_name
FROM event e
JOIN raum r ON e.raumid = r.raumnummer
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
WHERE e.datum = CURRENT_DATE
ORDER BY r.raumnummer, e.uhrzeit;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['raum_art', 'raumnummer', 'uhrzeit', 'patient_name']
('Untersuchungsraum', 4, datetime.time(0, 57, 22), 'Gertrude Schenk')
('Patientenzimmer', 7, datetime.time(1, 9, 35), 'Mirjana Bühlmann')
('Patientenzimmer', 7, datetime.time(19, 36, 36), 'Ursula Meyer')
('Patientenzimmer', 7, datetime.time(21, 6, 48), 'Carola Bauer')
('Patientenzimmer', 8, datetime.time(21, 15, 38), 'Till Iten')
('Patientenzimmer', 8, datetime.time(23, 52, 34), 'Ursula Meyer')
('Aufenthaltsraum', 10, datetime.time(8, 13, 33), 'Silas Ammann')
('Aufenthaltsraum', 10, datetime.time(17, 58, 59), 'Susi Stauffer')
('Untersuchungsraum', 13, datetime.time(10, 58, 42), 'Margaretha Albrecht')
('Untersuchungsraum', 13, datetime.time(15, 20, 50), 'Liam Lustenberger')
('Patientenzimmer', 16, datetime.time(4, 5, 12), 'Hans-Jürgen Jost')
('Patientenzimmer', 16, datetime.time(5, 25, 51), 'Marius Sidler')
('Patientenzimmer', 18, datetime.time(18, 29, 53), 'Liam Lustenberger')
('Aufenthaltsraum', 19, datetime.time(15, 52, 39), 'Jasi